Для  нашей задачи, крайне важно  правильно сформировать векторную базу данных, к сожалению, на тестовом датасете и на тестовых пример я получил некоторые ошибки и не точности
1) Очень кривое разбиение на чанки
2) Очень трудно разбивать и форматировать  ноды после того, как мы их запустили в убогом видем в базу данных
3) Нужно иметь четкий и человекочитаймый  код , а также название нодов в бд 
4) Не имеем станадрта pdf файлов, проблему почти не реально решить,  можно попробовать написать что-то похож на скрипт отчистки предусматривающий множество  вариантов


Тогда, как результат, тут попробуем сделать кранйе четкое и понятное формирование базы данных, с понятным раделением на тикиты, так же предлагаю усложнить самое рабочее пространство пуцтем добавления новых заивимостей

Наш стек:
1) LangChain
2) GigaChat
3) Neo4j
4) Numpay/Pandas
5) NTLK

Формирование корректного pdf файла с корректной и понтяной разбивкой чанков
Задачи:
1) Протись по каждой строке проверив ее корректность
2) ОБрезать  хотя бы до предложения, а не как раньше
3) Может быть сразу попробовать собрать какую-то ключевуб информацию?

In [8]:
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from nltk.tokenize import sent_tokenize

def clean_text(text):
    """
    Очищает текст от мусорных символов и нормализует пробелы.
    """

    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s\.,!?;:()\-—\[\]{}"\']', '', text)
    
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    text = re.sub(r'([.,!?;:])\s+', r'\1 ', text)
    
    return text.strip()

def is_valid_line(line):
    """
    Проверяет, является ли строка валидной (не мусорной).
    """

    if len(line.strip()) < 3:
        return False
    if not re.search(r'[а-яА-Яa-zA-Z0-9]', line):
        return False
    if re.fullmatch(r'[^а-яА-Яa-zA-Z0-9]+', line):
        return False
    
    return True

def clean_document_pages(documents):
    """
    Очищает каждую страницу документа, убирая мусор и проверяя строки.
    """
    cleaned_docs = []
    
    for doc in documents:
        if hasattr(doc, 'page_content'):
            text = doc.page_content
            metadata = doc.metadata.copy() if hasattr(doc, 'metadata') else {}
        else:
            text = doc
            metadata = {}
        
        lines = text.split('\n')

        cleaned_lines = []
        for line in lines:
            line = line.strip()
            if is_valid_line(line):
                cleaned_line = clean_text(line)
                if cleaned_line:
                    cleaned_lines.append(cleaned_line)
        
        
        # Собираем обратно
        if cleaned_lines:
            cleaned_text = ' '.join(cleaned_lines)
            cleaned_docs.append(Document(
                page_content=cleaned_text,
                metadata=metadata
            ))
    
    return cleaned_docs

def split_by_sentences(documents, max_sentences_per_chunk=10, sentence_overlap=2):
    """
    Разбивает документы на чанки по предложениям, не обрезая их.
    """
    all_chunks = []
    
    for doc in documents:
        text = doc.page_content
        metadata = doc.metadata.copy()
        
        # Разбиваем на предложения
        sentences = sent_tokenize(text, language='russian')
        
        # Если предложений мало, оставляем как есть
        if len(sentences) <= max_sentences_per_chunk:
            all_chunks.append(doc)
            continue
        
        # Разбиваем на чанки с перекрытием
        for i in range(0, len(sentences), max_sentences_per_chunk - sentence_overlap):
            chunk_sentences = sentences[i:i + max_sentences_per_chunk]
            if chunk_sentences:
                chunk_text = ' '.join(chunk_sentences)
                chunk_metadata = metadata.copy()
                chunk_metadata['sentence_start'] = i
                chunk_metadata['sentence_end'] = i + len(chunk_sentences) - 1
                chunk_metadata['total_sentences'] = len(sentences)
                
                all_chunks.append(Document(
                    page_content=chunk_text,
                    metadata=chunk_metadata
                ))
    
    return all_chunks

def split_by_pages(documents):
    """
    Разбивает документы по страницам (каждая страница - отдельный чанк).
    """
    # В вашем случае документы уже загружены по страницам
    # Здесь мы просто убеждаемся, что у каждой страницы есть правильные метаданные
    page_chunks = []
    
    for i, doc in enumerate(documents):
        metadata = doc.metadata.copy() if hasattr(doc, 'metadata') else {}
        
        # Добавляем/обновляем информацию о странице
        if 'page' not in metadata:
            metadata['page'] = i + 1
        
        page_chunks.append(Document(
            page_content=doc.page_content,
            metadata=metadata
        ))
    
    return page_chunks

def smart_split_documents(documents, chunk_size=1000, chunk_overlap=200, strategy='smart'):
    """
    Умное разбиение документов с выбором стратегии.
    
    Стратегии:
    - 'smart': автоматически выбирает лучший способ
    - 'pages': разбивает по страницам
    - 'sentences': разбивает по предложениям
    - 'recursive': использует RecursiveCharacterTextSplitter
    """
    
    # Сначала очищаем документы
    cleaned_docs = clean_document_pages(documents)
    
    if strategy == 'pages':
        return split_by_pages(cleaned_docs)
    
    elif strategy == 'sentences':
        return split_by_sentences(cleaned_docs)
    
    elif strategy == 'smart':

        # Автоматически выбираем стратегию на основе текста
        all_text = ' '.join([doc.page_content for doc in cleaned_docs])
        
        # Проверяем, есть ли четкое разделение по страницам
        if len(cleaned_docs) > 1 and all('page' in doc.metadata for doc in cleaned_docs):
            print("📄 Использую разбиение по страницам")
            return split_by_pages(cleaned_docs)
        
        # Иначе пробуем разбить по предложениям
        print("🔤 Использую разбиение по предложениям")
        return split_by_sentences(cleaned_docs)
    
    else:  # 'recursive' или по умолчанию
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""],  # Сохраняем предложения
            length_function=len,
        )
        return text_splitter.split_documents(cleaned_docs)

def load_and_prepare_pdfs(file_paths, max_pages=None, chunk_strategy='smart'):
    """
    Загружает PDF файлы, очищает и разбивает на чанки.
    """
    all_documents = []
    
    for file_path in file_paths:
        if file_path.endswith('.pdf'):
            print(f"📖 Загружаю PDF: {file_path}")
            loader = PyPDFLoader(file_path)
            loaded_docs = loader.load()
            
            if max_pages is not None and max_pages > 0:
                loaded_docs = loaded_docs[:max_pages]
                print(f"📄 Ограничено {max_pages} страницами")
            
            print(f"🧹 Очищаю текст...")
            cleaned_docs = clean_document_pages(loaded_docs)
            
            print(f"✂️ Разбиваю на чанки (стратегия: {chunk_strategy})...")
            chunks = smart_split_documents(cleaned_docs, strategy=chunk_strategy)
            
            all_documents.extend(chunks)
            
            print(f"✅ Обработано: {len(loaded_docs)} страниц -> {len(chunks)} чанков")
            print(f"   Пример чанка ({len(chunks[0].page_content) if chunks else 0} символов):")
            if chunks and len(chunks[0].page_content) > 100:
                print(f"   '{chunks[0].page_content[:100]}...'")
            print()
    
    # Статистика
    total_chars = sum(len(doc.page_content) for doc in all_documents)
    avg_chunk_size = total_chars / len(all_documents) if all_documents else 0
    
    print("=" * 60)
    print("📊 СТАТИСТИКА ОБРАБОТКИ:")
    print(f"   Всего документов/чанков: {len(all_documents)}")
    print(f"   Всего символов: {total_chars:,}")
    print(f"   Средний размер чанка: {avg_chunk_size:.0f} символов")
    print(f"   Минимальный размер: {min(len(doc.page_content) for doc in all_documents) if all_documents else 0}")
    print(f"   Максимальный размер: {max(len(doc.page_content) for doc in all_documents) if all_documents else 0}")
    print("=" * 60)
    
    return all_documents

# Пример использования
if __name__ == "__main__":
    # Тестируем на одном файле
    file_paths = ["../book2.pdf"]
    
    # Вариант 1: Постраничное разбиение (сохраняет страницы)
    print("🎯 Вариант 1: Разбиение по страницам")
    docs_by_pages = load_and_prepare_pdfs(
        file_paths,
        max_pages=10,  # Только первые 10 страниц для теста
        chunk_strategy='pages'
    )
    
    # Вариант 2: По предложениям (не обрезает предложения)
    print("\n🎯 Вариант 2: Разбиение по предложениям")
    docs_by_sentences = load_and_prepare_pdfs(
        file_paths,
        max_pages=10,
        chunk_strategy='sentences'
    )
    
    # Вариант 3: Умное разбиение (автоматический выбор)
    print("\n🎯 Вариант 3: Умное разбиение")
    docs_smart = load_and_prepare_pdfs(
        file_paths,
        max_pages=10,
        chunk_strategy='smart'
    )


🎯 Вариант 1: Разбиение по страницам
📖 Загружаю PDF: ../book2.pdf
📄 Ограничено 10 страницами
🧹 Очищаю текст...
✂️ Разбиваю на чанки (стратегия: pages)...
✅ Обработано: 10 страниц -> 10 чанков
   Пример чанка (219 символов):
   'Алексанор КРИВИЛЕВ КОМПЬЮТЕРНОЙ МАТЕМАТИКИ з фупсйол [ сисЁе[г,хО,у0) Ко. СВСЕ создание окружности г...'

📊 СТАТИСТИКА ОБРАБОТКИ:
   Всего документов/чанков: 10
   Всего символов: 15,411
   Средний размер чанка: 1541 символов
   Минимальный размер: 219
   Максимальный размер: 2319

🎯 Вариант 2: Разбиение по предложениям
📖 Загружаю PDF: ../book2.pdf
📄 Ограничено 10 страницами
🧹 Очищаю текст...
✂️ Разбиваю на чанки (стратегия: sentences)...
✅ Обработано: 10 страниц -> 29 чанков
   Пример чанка (219 символов):
   'Алексанор КРИВИЛЕВ КОМПЬЮТЕРНОЙ МАТЕМАТИКИ з фупсйол [ сисЁе[г,хО,у0) Ко. СВСЕ создание окружности г...'

📊 СТАТИСТИКА ОБРАБОТКИ:
   Всего документов/чанков: 29
   Всего символов: 17,888
   Средний размер чанка: 617 символов
   Минимальный размер: 102
   М